In [26]:
import json
import os
import pandas as pd
import ray

from pprint import pprint 

from env import QdTreeEnv
from qdtree import Workload
from qdtree.schema import ensure_data_schema
from policy import QdTreePolicy

from ray.rllib.env.env_context import EnvContext
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray import air, tune


Parameters

In [14]:
# We use sf = 1 for TPC-H. The paper uses sf = 1000
BASE_PATH = "../data/tpc-h/"
# The sampling rate in the paper is 0.1
SAMPLING_RATE = 0.01
# The min block size in the paper is 100K, we use 100 to scale down according to the sf
MIN_BLOCK_SIZE = 100 * SAMPLING_RATE

# Load the workload

In [18]:
WORKLOAD_PATH = os.path.join(BASE_PATH, "workload.json")
with open(WORKLOAD_PATH, "r") as f:
    workload = Workload(json.load(f))

workload._queries

{'1.4.sql': l_shipdate <= 906350400,
 '1.3.sql': l_shipdate <= 907041600,
 '1.2.sql': l_shipdate <= 902548800,
 '1.1.sql': l_shipdate <= 903240000,
 '1.5.sql': l_shipdate <= 905659200,
 '3.2.sql':  and(o_orderdate < 795070800, l_shipdate > 795070800),
 '3.5.sql':  and(o_orderdate < 796626000, l_shipdate > 796626000),
 '3.4.sql':  and(o_orderdate < 795243600, l_shipdate > 795243600),
 '3.3.sql':  and(o_orderdate < 796453200, l_shipdate > 796453200),
 '3.1.sql':  and(o_orderdate < 796280400, l_shipdate > 796280400),
 '4.4.sql':  and(o_orderdate >= 794034000, o_orderdate < 801979200),
 '4.1.sql':  and(o_orderdate >= 854773200, o_orderdate < 862459200),
 '4.2.sql':  and(o_orderdate >= 783666000, o_orderdate < 791614800),
 '4.5.sql':  and(o_orderdate >= 875678400, o_orderdate < 883630800),
 '4.3.sql':  and(o_orderdate >= 865137600, o_orderdate < 873086400)}

# Load and sample data

In [19]:
DATA_PATH = os.path.join(BASE_PATH, "sf1/denormalized.parquet")
all_data = pd.read_parquet(DATA_PATH).sample(frac=0.1)
all_data.columns = all_data.columns.str.lower()
all_data = ensure_data_schema(all_data, workload.schema)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600122 entries, 1214116 to 3540802
Data columns (total 35 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   l_orderkey        600122 non-null  int64  
 1   l_partkey         600122 non-null  int64  
 2   l_suppkey         600122 non-null  int64  
 3   l_linenumber      600122 non-null  int64  
 4   l_quantity        600122 non-null  float64
 5   l_extendedprice   600122 non-null  float64
 6   l_discount        600122 non-null  float64
 7   l_tax             600122 non-null  float64
 8   l_shipdate        600122 non-null  int64  
 9   l_commitdate      600122 non-null  int64  
 10  l_receiptdate     600122 non-null  int64  
 11  o_orderkey        600122 non-null  int64  
 12  o_custkey         600122 non-null  int64  
 13  o_totalprice      600122 non-null  float64
 14  o_orderdate       600122 non-null  int64  
 15  o_shippriority    600122 non-null  int64  
 16  c_custkey    

In [20]:
data = all_data.sample(frac=SAMPLING_RATE)
len(data)

6001

# Set up the environment

In [12]:
ray.init(local_mode=False, ignore_reinit_error=True)

2023-05-14 14:21:14,320	INFO worker.py:1382 -- Calling ray.init() again after it has already been called.


Python version:,3.8.10
Ray version:,2.3.1


In [21]:
env_config = {
    "workload": workload,
    "data": data,
    "min_leaf_size": MIN_BLOCK_SIZE,
}

config = (
    PPOConfig()
    .environment(
        QdTreeEnv,
        env_config=env_config,
    )
    .framework("torch")
    .rollouts(num_rollout_workers=5, batch_mode="complete_episodes")
    .training(
        model={
            "fcnet_hiddens": [512, 512],
            "fcnet_activation": "relu",
            # "vf_share_layers": True,
        }
    )
    .multi_agent(
        policies={DEFAULT_POLICY_ID: PolicySpec(policy_class=QdTreePolicy)}
    )
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
)

algo = config.build()

stop = {
    "training_iteration": 10,
    "timesteps_total": 40000,
    "episode_reward_mean": 0.95,
}

(RolloutWorker pid=28675) 2023-05-14 14:30:47,445	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=28675) 2023-05-14 14:30:47,445	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
2023-05-14 14:30:47,875	WARNING util.py:67 -- Install gputil for GPU system monitoring.


In [22]:
tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(
        local_dir="./results",
        stop=stop,
        checkpoint_config=air.CheckpointConfig(checkpoint_at_end=True),
    ),
)
result = tuner.fit()

(PPO pid=29201) 2023-05-14 14:31:09,448	WARNING algorithm_config.py:596 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=29201) 2023-05-14 14:31:09,650	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(PPO pid=29201) 2023-05-14 14:31:13,599	WARNING util.py:67 -- Install gputil for GPU system monitoring.


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_QdTreeEnv_7d18b_00000,12072,"{'ObsPreprocessorConnector_ms': 0.014071578071230934, 'StateBufferConnector_ms': 0.009673266183762323, 'ViewRequirementAgentConnector_ms': 0.20046012742178781}","{'num_env_steps_sampled': 12072, 'num_env_steps_trained': 12072, 'num_agent_steps_sampled': 12072, 'num_agent_steps_trained': 12072}",{},2023-05-14_14-33-32,True,9.60476,{},0.561806,0.309581,0.00149975,420,1358,de3bd72764b44c909ad7f630d582abf3,HOMELAB,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 13.553956789483307, 'cur_kl_coeff': 0.4500000000000001, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 2.2576730605094664, 'policy_loss': -0.04013065214429091, 'vf_loss': 2.2893530917424028, 'vf_explained_var': 0.10801456532170696, 'kl': 0.018779171893011055, 'entropy': 3.082078647100797, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 2325.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 12072, 'num_env_steps_trained': 12072, 'num_agent_steps_sampled': 12072, 'num_agent_steps_trained': 12072}",3,192.168.75.74,12072,12072,12072,4034,12072,4034,0,5,0,0,4034,"{'cpu_util_percent': 6.5234375, 'ram_util_percent': 45.68906249999999}",29201,{},{},{},"{'mean_raw_obs_processing_ms': 0.6667887888119145, 'mean_inference_ms': 2.6279201217550527, 'mean_action_processing_ms': 0.1892432076968075, 'mean_env_wait_ms': 1.1887411672170634, 'mean_env_render_ms': 0.0}",138.3,45.162,138.3,"{'training_iteration_time_ms': 46086.698, 'load_time_ms': 0.965, 'load_throughput': 4171153.957, 'learn_time_ms': 42119.596, 'learn_throughput': 95.537, 'synch_weights_time_ms': 23.098}",1684089212,0,12072,3,7d18b_00000,3.96559


In [27]:
agent = config.build()
agent.restore(result.get_best_result().checkpoint) # type: ignore

env = QdTreeEnv(EnvContext(env_config, 0))
done = False
obs, _ = env.reset() 
step = 0
episode_reward = 0
while not done:
    action = agent.compute_single_action(obs, explore=False)
    obs, reward, done, _, info = env.step(action) # type: ignore
    episode_reward += reward
    # print(step, action, obs, reward, done, info)
    step += 1

pprint(env.qd_tree.blocks)
print(episode_reward)


(RolloutWorker pid=32265) 2023-05-14 14:38:45,464	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=32265) 2023-05-14 14:38:45,464	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
2023-05-14 14:38:45,740	WARNING util.py:67 -- Install gputil for GPU system monitoring.
2023-05-14 14:38:45,837	INFO trainable.py:791 -- Restored on 192.168.75.74 from checkpoint: /home/ctring/src/qdtree/src/results/PPO/PPO_QdTreeEnv_7d18b_00000_0_2023-05-14_14-31-06/checkpoint_000003
2023-05-14 14:38:45,838	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 3, '_timesteps_total': None, '_time_total': 138.29967379570007, '_episodes_total': 1358}


[{'c_acctbal': (-inf, inf),
  'c_custkey': (-inf, inf),
  'c_nationkey': (-inf, inf),
  'l_commitdate': (-inf, inf),
  'l_discount': (-inf, inf),
  'l_extendedprice': (-inf, inf),
  'l_linenumber': (-inf, inf),
  'l_orderkey': (-inf, inf),
  'l_partkey': (-inf, inf),
  'l_quantity': (-inf, inf),
  'l_receiptdate': (-inf, inf),
  'l_shipdate': (-inf, inf),
  'l_suppkey': (-inf, inf),
  'l_tax': (-inf, inf),
  'n_nationkey_cust': (-inf, inf),
  'n_nationkey_supp': (-inf, inf),
  'n_regionkey_cust': (-inf, inf),
  'n_regionkey_supp': (-inf, inf),
  'o_custkey': (-inf, inf),
  'o_orderdate': [854773200, 875678400),
  'o_orderkey': (-inf, inf),
  'o_shippriority': (-inf, inf),
  'o_totalprice': (-inf, inf),
  'p_partkey': (-inf, inf),
  'p_retailprice': (-inf, inf),
  'p_size': (-inf, inf),
  'ps_availqty': (-inf, inf),
  'ps_partkey': (-inf, inf),
  'ps_suppkey': (-inf, inf),
  'ps_supplycost': (-inf, inf),
  'r_regionkey_cust': (-inf, inf),
  'r_regionkey_supp': (-inf, inf),
  's_acctbal'

(raylet) [2023-05-14 14:47:29,558 E 25473 25473] (raylet) node_manager.cc:3040: 6 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 9790178f757169f649542af646f31498807e6faa2523246e90be57ae, IP: 192.168.75.74) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.75.74`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(RolloutWorker pid=2021) 2023-05-14 14:47:57,902	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_st

In [ ]:
print(env.qd_tree)